In [2]:
import requests # 웹 요청 도구
from bs4 import BeautifulSoup # html에서 데이터 읽는 도구
from bs4.element import NavigableString, Tag, Comment, ProcessingInstruction

In [3]:
response = requests.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findRealTicketList.do")

In [4]:
soup = BeautifulSoup(response.content)

In [5]:
table = soup.select_one('.tbl_comm.th_sort')

In [118]:
th_list = table.select('thead tr th')
columns = []
for th in th_list:
    for c in th.children:
        # print(c)
        if type(c) == NavigableString and len(c.strip()) > 0:
            columns.append(c.strip())

In [119]:
columns

['순위', '영화명', '개봉일', '예매율', '예매매출액', '누적매출액', '예매관객수', '누적관객수']

In [120]:
tr_list = table.select('tbody tr')
values_list = []
for tr in tr_list:
    for td in tr.select('td'):
        print(td)


#beautifulsoup만으로는 크롤링 불가능. selenium(+chrome webdriver) 설치 필수

<td colspan="8">검색된 데이터가 존재하지 않습니다.</td>


In [14]:
from selenium import webdriver

In [15]:
browser_proxy = webdriver.Chrome("/Users/attitude/workspace/tools/chromedriver")

/opt/anaconda3/envs/kdigital-pusan/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [16]:
browser_proxy.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findRealTicketList.do")

In [9]:
soup = BeautifulSoup(browser_proxy.page_source)

In [10]:
table = soup.select_one('.tbl_comm.th_sort')

In [11]:
th_list = table.select('thead tr th')
columns = []
for th in th_list:
    for c in th.children:
        # print(c)
        if type(c) == NavigableString and len(c.strip()) > 0:
            columns.append(c.strip())

In [12]:
columns

['순위', '영화명', '개봉일', '예매율', '예매매출액', '누적매출액', '예매관객수', '누적관객수']

In [13]:
tr_list = table.select('tbody tr')
values_list = []
for tr in tr_list:
    values = []
    for td in tr.select('td'):
        values.append(td.text.strip().replace("%", '').replace(",", ""))
    values_list.append(values)


In [129]:
values_list[:3]

[['1',
  '듄',
  '2021-10-20',
  '15.7',
  '434880110',
  '15367958720',
  '26516',
  '1427720'],
 ['2',
  '고스트버스터즈 라이즈',
  '2021-12-01',
  '13.4',
  '199034210',
  '5115000',
  '22574',
  '545'],
 ['3',
  '유체이탈자',
  '2021-11-24',
  '10.7',
  '149559470',
  '3871631940',
  '18131',
  '398457']]

In [14]:
import pandas as pd

data = pd.DataFrame(values_list, columns=columns)

In [15]:
# data.info()
type(data.dtypes)
for column in data.columns[[0, 3, 4, 5, 6, 7]]:
    data[column] = data[column].astype(float)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   순위      252 non-null    float64
 1   영화명     252 non-null    object 
 2   개봉일     252 non-null    object 
 3   예매율     252 non-null    float64
 4   예매매출액   252 non-null    float64
 5   누적매출액   252 non-null    float64
 6   예매관객수   252 non-null    float64
 7   누적관객수   252 non-null    float64
dtypes: float64(6), object(2)
memory usage: 15.9+ KB


In [17]:
data.head(10)

,순위,영화명,개봉일,예매율,예매매출액,누적매출액,예매관객수,누적관객수
0,1.0,듄,2021-10-20,15.7,434030010.0,1.536796e+10,26429.0,1427720.0
1,2.0,고스트버스터즈 라이즈,2021-12-01,13.5,200069710.0,5.115000e+06,22688.0,545.0
2,3.0,유체이탈자,2021-11-24,10.7,148648200.0,3.871632e+09,18044.0,398457.0
3,4.0,라스트 나잇 인 소호,2021-12-01,9.5,127832510.0,2.358000e+06,16067.0,262.0
4,5.0,연애 빠진 로맨스,2021-11-24,7.0,107459930.0,2.338057e+09,11768.0,231647.0
5,6.0,엔칸토: 마법의 세계,2021-11-24,5.9,92065600.0,2.718391e+09,9961.0,284868.0
6,7.0,태일이,2021-12-01,5.2,72067930.0,4.826570e+08,8747.0,58523.0
7,8.0,킬링 카인드: 킬러의 수제자,2021-12-01,3.6,48586730.0,3.756000e+06,6114.0,406.0
8,9.0,몬스타엑스 : 더 드리밍,2021-12-08,2.8,75984000.0,0.000000e+00,4749.0,0.0
9,10.0,귀멸의 칼날: 주합회의·나비저택 편,2021-12-01,2.7,40266270.0,0.000000e+00,4468.0,0.0


In [19]:
data.groupby(by=["순위"])["순위"].count()

순위
1.0       1
2.0       1
3.0       1
4.0       1
5.0       1
         ..
182.0     6
188.0    13
201.0    17
218.0    20
238.0    15
Name: 순위, Length: 124, dtype: int64

In [21]:
import pymysql
import pandas as pd
import os

In [24]:
# csv파일 DB(mysql)에 저장  ->   510,392개 데이터

import numpy as np

# 1. connect
conn = pymysql.connect(host="127.0.0.1", port=3306, db='finalteam3', user="kdigital", password="mysql", charset="utf8")

# 2. get command object
cursor = conn.cursor()
#cursor.execute("delete from movie") #기존 데이터를 지울 때 주석을 풀어주세요 

# 3. execute command
sql = """insert into movie_ticketing (ranked, title, release_date, reserve_rate, reserve_revenue, accumulated_revenue, ticketing_view, accumulated_view) 
         values (%s, %s, %s, %s, %s, %s, %s, %s)"""
for movie in data.values:
    cursor.execute(sql, list(movie))
                           
# break

conn.commit() # confirm previous executio

# 4. close resource
cursor.close()
conn.close()

In [20]:
# browser_proxy.close() #주석 풀어서 사용하세요) 가장 마지막 크롬 하나만 종료
# browser_proxy.quit() #주석 풀어서 사용하세요) 크롬s 모두 종료